# Using AIfES Converter
Run this notebook after executing TF-Logic-Gates-Demo.json.

For more detail on AIfES see:
 - https://github.com/Fraunhofer-IMS/AIfES-Converter
 - https://fraunhofer-ims.github.io/AIfES-Converter/quickstart.html
 - https://github.com/Fraunhofer-IMS/AIfES_for_Arduino

Ensure Python 3.9 for AIfES

In [1]:
import sys
sys.version

'3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]'

In [2]:
import numpy as np
from pathlib import Path
from tensorflow import keras
from aifes import keras2aifes

Network input data, required to set quanitzation range

In [3]:
representative_dataset = np.array([[0, 0],
                                   [0, 1],
                                   [1, 0],
                                   [1, 1]])

List of models

In [4]:
models_list = [
    'and-gate',
    'nor-gate',
    'or-gate',
    'xnor-gate',
    'xor-gate',
    
]

Load a model from the model list, use AIfESConverter and create a float and a q7 model. Parse the generated c files and exchange the generic variable, function and define names with model specific ones.

In [5]:
for model in models_list:
    keras_model = keras.models.load_model(model + '.h5')
    dest_path = Path('..\\aifes-gates-cube-project\\AIfES-models\\AIfES-')
    model_path = Path(str(dest_path) + model + '-float')
    model_name = model.replace('-', '_')
    keras2aifes.convert_to_fnn_f32_express(keras_model, str(model_path))
    
    # Adjust the name of the float model
    # AIfES always uses the same model name, this section changes the name of the model
    # and makes it possible to import multiple models within the same c code file.
    fnn_h = model_path / 'aifes_e_f32_fnn.h'
    with open(fnn_h,'r') as file:
        filedata = file.read()
        # Adjust function names
        filedata = filedata.replace('aifes_e_f32_fnn_inference(','aifes_e_' + model_name + '_f32_fnn_inference(')
        # Adjust variable names
        filedata = filedata.replace('(void*)aifes_e_f32_flat_weights;','(void*) model_' + model_name + '_flat_weights;')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_F32_FNN','AIFES_E_F32_' + model_name.upper() + '_FNN')
    with open(fnn_h,'w') as file:
        file.write(filedata)
    
    weights_h = model_path / 'aifes_e_f32_weights.h'
    with open(weights_h,'r') as file:
        filedata = file.read()
        # Adjust variable names
        filedata = filedata.replace('aifes_e_f32_flat_weights[]','model_' + model_name + '_flat_weights[]')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_F32_WEIGHTS','AIFES_E_F32_' + model_name.upper() + '_WEIGHTS')
    with open(weights_h,'w') as file:
        file.write(filedata)
    
    
    model_path = Path(str(dest_path) + model + '-q7')
    # Quantized Q7 model with 32 bit storage alignment (4 Byte) and little endian representation
    keras2aifes.convert_to_fnn_q7_express(keras_model, str(model_path), 
                                      representative_data=representative_dataset, 
                                      target_alignment=4, 
                                      byteorder="little")
    
    
    # Adjust the name of the q7 model
    # AIfES always uses the same model name, this section changes the name of the model
    # and makes it possible to import multiple models within the same c code file.
    fnn_h = model_path / 'aifes_e_q7_fnn.h'
    with open(fnn_h,'r') as file:
        filedata = file.read()
        # Adjust function names
        filedata = filedata.replace('aifes_e_q7_fnn_inference(','aifes_e_' + model_name + '_q7_fnn_inference(')
        # Adjust variable names
        filedata = filedata.replace('(void*) model_parameters;','(void*) model_' + model_name + '_parameters;')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_Q7_FNN','AIFES_E_Q7_' + model_name.upper() + '_FNN')
    with open(fnn_h,'w') as file:
        file.write(filedata)
    
    weights_h = model_path / 'aifes_e_q7_weights.h'
    with open(weights_h,'r') as file:
        filedata = file.read()
        # Adjust variable names
        filedata = filedata.replace('model_parameters[','model_' + model_name + '_parameters[')
        filedata = filedata.replace('parameter_memory_size','parameter_' + model_name + '_memory_size')
        # Adjust defines
        filedata = filedata.replace('AIFES_E_Q7_WEIGHTS','AIFES_E_Q7_' + model_name.upper() + '_WEIGHTS')
    with open(weights_h,'w') as file:
        file.write(filedata)
    